In [40]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
from glob import glob

import pandas as pd
import numpy as np
import dill

from maze_io import load_maze, string_maze, add_padding
from maze_env import MazeEnv, MazeAction
from agent import Agent, manhattan, euclid, zero_heuristic
from navmesh_agent import NavmeshAgent
from agent_utils import Stats
from general_utils import get_optimal_path_length

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
def run_single_maze(start_loc, exit_loc, maze, agentClass, heuristic, agent_verbose=False) -> Stats:
    env = MazeEnv(start_loc, exit_loc, maze)
    agent = agentClass(env, heuristic=heuristic, verbose=agent_verbose)
    agent.run()
    return agent.statistics

def prepare_maze_from_file(path):
    with path.open(mode='r') as f:
        padded_maze_tuple = add_padding(*load_maze(f))
    return padded_maze_tuple

In [14]:
n_times = 100  # how many times to repeat each maze-agent settings combination

In [ ]:
inpaths = list(map(Path, glob('./mazes/*.txt')))
maze_names = [path.stem for path in inpaths]

# Compute A* Agent statistics
Beware, takes a very long time (2h+)

In [24]:
data = {}
heuristics = {'None': zero_heuristic,
              'Euclid': euclid,
              'Manhattan': manhattan}
for maze_num, path in enumerate(inpaths, start=1):
    maze_name = path.stem
    maze_data = prepare_maze_from_file(path)
    optimal_length = get_optimal_path_length(*maze_data)
    data[maze_name] = {"optimal": optimal_length}
    for heuristic_name, heuristic in heuristics.items():
        print(f'\r#{maze_num}/{len(inpaths)}: {maze_name} | {heuristic_name}         ', end="")
        np.random.seed(123)
        data[maze_name][heuristic_name] = [run_single_maze(*maze_data, Agent, heuristic=heuristic, agent_verbose=False)
                                           for n in range(n_times)]
print("\nDone")

#25/25: maze-7-5 | Manhattan          
Done


### Optional saving and loading of A* raw data
To avoid overwriting files when run all cells is used, simple if False blocks are used, change to True if you want to save or load

In [32]:
if False:
    with open('./a_star_data.dill', mode='wb') as f:
        dill.dump(data, f)

In [80]:
if False:
    with open('./a_star_data.dill', mode='rb') as f:
        astar_data = dill.load(f)

## Create A* dataframe

In [81]:
dfs = []
for name in maze_names:
    cur_opt = astar_data[name]["optimal"]
    for heur, cur_stats in astar_data[name].items():
        if heur != "optimal":
            tuplized = [(np.sum(stats.expanded_nodes),
                         stats.replan_count,
                         stats.steps)
                        for stats in cur_stats]
            cur_expanded, cur_replans, cur_steps = zip(*tuplized)
            cur_dict = {"maze": name,
                        "heuristic": heur,
                        "optimal_steps": cur_opt,
                        "steps_taken": cur_steps,
                        "expanded_nodes": cur_expanded,
                        "replan_count": cur_replans}
            dfs.append(pd.DataFrame(cur_dict))
astar_df = pd.concat(dfs)

# Create Navmesh Agent statistics
Is fairly quick, under a minute

In [83]:
data_navmesh = {}
optimal_paths = {}
for maze_num, path in enumerate(inpaths, start=1):
    maze_name = path.stem
    maze_data = prepare_maze_from_file(path)
    optimal_length = get_optimal_path_length(*maze_data)
    optimal_paths[maze_name] = optimal_length
    print(f'\r#{maze_num}/{len(inpaths)}: {maze_name}         ', end="")
    np.random.seed(123)
    data_navmesh[maze_name] = [run_single_maze(*maze_data, NavmeshAgent, heuristic=None, agent_verbose=False)
                               for n in range(n_times)]
print("\nDone")

#25/25: maze-7-5          
Done


## Create Navmesh Agent dataframe

In [45]:
dataframes = []
for name in maze_names:
    cur_maze_stats = data_navmesh[name]
    cur_opt = optimal_paths[name]
    tuplized = [(np.sum(stats.expanded_nodes),
                 1,
                 stats.steps) for stats in cur_maze_stats]
    cur_expanded, cur_replans, cur_steps = zip(*tuplized)
    cur_dict = {"maze": name,
                "optimal_steps": cur_opt,
                "steps_taken": cur_steps,
                "expanded_nodes": cur_expanded,
                "replan_count": cur_replans}
    dataframes.append(pd.DataFrame(cur_dict))
nav_df = pd.concat(dataframes)

# Various statistics

In [73]:
nav_df.groupby('maze').agg(optimal_steps=('optimal_steps', max),
                           dist_to_opt=('dist_to_opt', np.mean),
                           mean_steps=('steps_taken', np.mean),
                           std_steps=('steps_taken', np.std),
                           min_steps=('steps_taken', min),
                           max_steps=('steps_taken', max),
                           expanded_nodes=('expanded_nodes', max),
                           replan_count=('replan_count', max)).reset_index()


,maze,optimal_steps,dist_to_opt,mean_steps,std_steps,min_steps,max_steps,expanded_nodes,replan_count
0,maze-12-1,46,39.92,85.92,13.406376,61,123,339,1
1,maze-12-2,46,38.73,84.73,11.640972,57,126,350,1
2,maze-12-3,54,45.55,99.55,11.388790,70,130,356,1
3,maze-12-4,46,39.38,85.38,11.559053,61,119,356,1
4,maze-12-5,46,36.08,82.08,11.619107,58,110,351,1
5,maze-25-1,98,87.55,185.55,20.641419,146,246,1534,1
6,maze-25-2,104,91.19,195.19,18.593849,163,258,1556,1
7,maze-25-3,98,79.78,177.78,15.849405,146,209,1547,1
8,maze-25-4,98,84.17,182.17,17.835418,144,232,1542,1
9,maze-25-5,104,93.60,197.60,19.645340,157,247,1548,1


In [82]:
astar_df.groupby(['maze', 'heuristic']).mean()

optimal_steps  steps_taken  expanded_nodes  replan_count
maze      heuristic                                                          
maze-12-1 Euclid              46.0        84.06         1519.19         12.40
          Manhattan           46.0        84.47          889.01         12.59
          None                46.0        84.75         2905.15         12.28
maze-12-2 Euclid              46.0        84.41         1801.94         14.56
          Manhattan           46.0        82.54          949.81         13.86
...                            ...          ...             ...           ...
maze-7-4  Manhattan           26.0        50.87          301.17          9.23
          None                26.0        50.40          786.08          8.80
maze-7-5  Euclid              26.0        47.59          474.96          8.96
          Manhattan           26.0        46.98          378.08          8.16
          None                26.0        48.11          816.10          8.49

[75 rows x 4 columns]